In [5]:
import PyPDF2
import re
import pathlib
import os
import pandas as pd
import tensorflow.keras as keras

In [2]:
def read_file_set(path):
    data = []
    for fn in (path).iterdir():
        with open(fn, 'rb') as file:
            read_pdf = PyPDF2.PdfFileReader(file)
            text = ""
            for i in range(read_pdf.getNumPages()):
                page = read_pdf.getPage(i)
                page_content = page.extractText()
                text += ''.join(page_content)
            #text = text.replace("Tribunal de Justiça de Minas Gerais","")
            #text = re.sub(".*Data da Publicação:(EMENTA: *)?", "", text)
            data.append(text)
    return data

In [3]:
dataset_dir = pathlib.Path(os.getcwd())
data = read_file_set(dataset_dir/"documentos")

In [4]:
df = pd.DataFrame(data, columns=["text"])
df

,text
0,Tribunal de Justiça de Minas Gerais1.0000.18.0...
1,Tribunal de Justiça de Minas Gerais1.0000.18.1...
2,Tribunal de Justiça de Minas Gerais1.0000.19.1...
3,Tribunal de Justiça de Minas Gerais1.0000.20.0...
4,Tribunal de Justiça de Minas Gerais1.0000.20.0...
5,Tribunal de Justiça de Minas Gerais1.0000.20.0...
6,Tribunal de Justiça de Minas Gerais1.0000.20.0...
7,Tribunal de Justiça de Minas Gerais1.0000.20.0...
8,Tribunal de Justiça de Minas Gerais1.0000.20.0...
9,Tribunal de Justiça de Minas Gerais1.0000.20.0...


In [5]:
df['code'] = df.text.str.extract("Número do(\d{7})-Númeraçã")

In [6]:
df.text = df.text.str.replace("Tribunal de Justiça de Minas Gerais","").str.replace(".*Data da Publicação:(EMENTA: *)?", "",regex=True)

In [7]:
df

,text,code
0,APELAÇÃO CÍVEL. AÇÃO DECLARATÓRIA DE INEXISTÊN...,5000215
1,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...,5005969
2,CIVIL. DECLARAÇÃO DE INEXISTÊNCIA DE DÉBITO.CO...,0018496
3,APELAÇÃO CÍVEL. AÇÃO INDENIZATÓRIA. SERVIÇO DE...,0036955
4,APELAÇÃO CÍVEL. CONEXÃO. FEITOS JULGADOS CONJU...,0014170
5,APELAÇÃO CÍVEL. AÇÃO DECLARATÓRIA DE INEXISTÊN...,0461813
6,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...,0490564
7,CONSUMIDOR. SERVIÇOS DE TELEFONIA. ALEGAÇÃO DE...,5001108
8,APELAÇÃO. AÇÃO DE REPARAÇÃO POR DANOS MATERIAI...,5005142
9,APELAÇÃO CÍVEL. AÇÃO DECLARATÓRIA DE INEXISTÊN...,5002401


In [8]:
resultados = pd.read_excel('cadastro.xlsx', usecols=["Autos","Resultado"], dtype = {"Resultado": "category"})
resultados = resultados.dropna()
resultados.Resultado = resultados.Resultado.values.codes
resultados.Autos = resultados.Autos.str.replace("[^\d]", "", regex=True).str[:7]
resultados

,Autos,Resultado
0,5000215,1
1,0014170,1
2,5005969,0
3,5002014,1
4,0461813,1
5,5008163,0
6,0036955,0
19,0490564,0
20,5005142,0
21,5013907,0


In [9]:
df = pd.merge(df, resultados, left_on="code", right_on="Autos")[["text", "Resultado"]]

In [10]:
df=df.sort_values(by="Resultado")

In [169]:
df.to_csv("./dataframe.csv", index=False)

In [194]:
df = pd.read_csv('./dataframe.csv')

In [195]:
df.columns=['text','Resultado']

In [196]:
df

,text,Resultado
0,APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS...,0
1,CONSUMIDOR. SERVIÇOS DE TELEFONIA. ALEGAÇÃO DE...,0
2,APELAÇÃO - CADASTRO RESTRITIVO DE CRÉDITO -INC...,0
3,APELAÇÃO CÍVEL - AÇÃO INDENIZATÓRIA - CONTRATO...,0
4,< APELAÇÃO CÍVEL - EMPRESA DE TELEFONIA - AÇÃO...,0
5,APELAÇÃO CÍVEL. AÇÃO INDENIZATÓRIA. SERVIÇO DE...,0
6,APELAÇÃO. AÇÃO DE REPARAÇÃO POR DANOS MATERIAI...,0
7,APELAÇÃO CÍVEL - AÇÃO ORDINÁRIA - INEXISTÊNCIA...,0
8,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...,0
9,APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS...,0


In [197]:
train_date = pd.concat([df[:df.shape[0]//4],df[df.shape[0]//2:3 * df.shape[0]//4]])
test_date = pd.concat([df[df.shape[0]//4:df.shape[0]//2],df[3 * df.shape[0]//4:]])

In [198]:
test_date

,text,Resultado
5,APELAÇÃO CÍVEL. AÇÃO INDENIZATÓRIA. SERVIÇO DE...,0
6,APELAÇÃO. AÇÃO DE REPARAÇÃO POR DANOS MATERIAI...,0
7,APELAÇÃO CÍVEL - AÇÃO ORDINÁRIA - INEXISTÊNCIA...,0
8,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...,0
9,APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS...,0
10,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...,0
16,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...,1
17,APELAÇÃO CÍVEL. AÇÃO DECLARATÓRIA DE INEXISTÊN...,1
18,APELAÇÃO CÍVEL - AÇÃO DECLARATÓRIA DE INEXISTÊ...,1
19,APELAÇÃO. CIVIL E PROCESSUAL CIVIL. GRATUIDADE...,1


# Classificação

In [199]:
entrada = keras.layers.TextVectorization(output_mode='tf_idf')
entrada.adapt(train_date.text)

In [200]:
model = keras.Sequential([
        entrada,
        keras.layers.Dense(8),
        keras.layers.Dense(8),
        keras.layers.Dense(1)
    ])

In [201]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy'])

In [202]:
history = model.fit(train_date.text,train_date.Resultado, epochs=5)

Epoch 1/5
1/1 [==============================] - 1s 758ms/step - loss: 3.4814 - binary_accuracy: 0.6000
Epoch 2/5
1/1 [==============================] - 0s 14ms/step - loss: 1.6577 - binary_accuracy: 0.8000
Epoch 3/5
1/1 [==============================] - 0s 12ms/step - loss: 1.5458 - binary_accuracy: 0.9000
Epoch 4/5
1/1 [==============================] - 0s 14ms/step - loss: 1.5255 - binary_accuracy: 0.9000
Epoch 5/5
1/1 [==============================] - 0s 11ms/step - loss: 1.5249 - binary_accuracy: 0.9000


In [203]:
binary_loss, binary_accuracy = model.evaluate(test_date.text,test_date.Resultado)
print("loss: %s\naccuracy: %s" % (binary_loss, binary_accuracy))

1/1 [==============================] - 0s 435ms/step - loss: 7.7125 - binary_accuracy: 0.5000
loss: 7.712474346160889
accuracy: 0.5
